# Iron Condor Options Strategy Guide

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Connecting to OpenAI (or Ollama)

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
# via OpenAI API
# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions
# openai = OpenAI()

# via ollama running locally
try:
    openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    print("Ollama object created successfully!")
except Exception as e:
    print(f"Error creating Ollama object: {e}")


Ollama object created successfully!


# Let's make a quick call to a Frontier model to get started, as a preview!

In [14]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.
# message = "Hello, GPT! This is my first ever message to you! Hi!"
# response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
# print(response.choices[0].message.content)

# To give you a preview -- calling ollama with these messages is this easy. Any problems, head over to the Troubleshooting notebook.
message = "Hello, Llama! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-oss:20b", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! 👋 It’s great to meet you! I’m Llama, ready to chat, answer questions, help brainstorm ideas, or just talk about whatever’s on your mind. What would you like to dive into today?


# OK onwards with our first project

In [16]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [17]:
# Let's try one out. Change the website and add print statements to follow along.

iron_condor_site = Website("https://optionalpha.com/strategies/iron-condor")
print(iron_condor_site.title)
print(iron_condor_site.text)

Iron Condor Options Strategy [Download Your Free Guide]
Get Option Alpha
100% FREE
by simply connecting your TradeStation or Tradier Brokerage account!
Learn more
.
📣 Our 1st LIVE event is scheduled!
Friday, October 3rd, in New York City! 👉
Learn more
.
Home
Resources
Bots 101
Integrations
Education
Courses
Strategies
Handbook
Podcast
Blog
Pricing
Brokers
FAQ
Start Free Trial
Login

Login
Start Free Trial
Home
Education
Options Strategies
Iron Condor
Iron Condor
Approved for autotrading
An iron condor is a multi-leg, risk-defined, neutral strategy with limited profit potential.
View risk disclosures
optionalpha.com/strategies/iron-condor
Learn
Stats
Templates
Iron Condor
overview
An iron condor consists of selling an out-of-the-money
bear call credit spread
above the stock price and an out-of-the-money
bull put credit spread
below the stock price with the same expiration date.
The strategy looks to take advantage of a drop in volatility, time decay, and little or no movement from the 

In [18]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [20]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

# And now let's build useful messages for LLM, using a function

In [21]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

# Time to bring it together - the API for OpenAI is very simple!

In [23]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        #model = "gpt-4o-mini",
        model = "gpt-oss:20b",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [24]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [25]:
display_summary("https://optionalpha.com/strategies/iron-condor")

# Iron Condor Options Strategy – Quick Overview

**What the page covers**

- **Strategy fundamentals** – definition, components (bull put credit spread + bear call credit spread), payoff diagram, market outlook, and risk/return profile.  
- **Setup & execution** – steps to open an iron condor, choosing strike distances, and the logic behind selling‑to‑open the credit spreads.  
- **Trade management** – ways to exit (full close, one‑leg close, buying back shorts), time‑decay (theta), and volatility (vega) benefits.  
- **Adjustments & rolls** – tactics such as rolling a leg up or down, extending expiration, or converting to an iron butterfly when the underlying moves.  
- **Risk limits** – clear max loss, break‑even points, and assignment considerations.  
- **Resources & community** – links to templates, bots, tutorials, podcast episodes, and a community of >11,000 traders.

**Announcements & news**

- **Live event:** First live session scheduled for **Friday, October 3, 2024** in New York City.  
- **Update timeline:** The guide was last updated **February 8 2025** (content originally published Dec 28 2020 / Feb 12 2021).  

The page is a concise, step‑by‑step reference for traders who want a neutral options play that benefits from low volatility, minimal price movement, and time decay.